# Chirality Check

Chirality is a symmetry property with great significance in chemistry, biology
and pharmaceutical science. Here we show how to determine if two compounds
are enantiomers by using rotational Procrustes. Suppose we have two compounds
with the same formula: CHFClBr and there is one chiral carbon atom in each of
them. The absolute configuration of the compounds are shown in the figure below. To determine whether the two compounds are chiral enantiomers, we compare the rms
distance between structure when only rotational transformations are allowed and
when both rotational transformations and reflection are allowed. If the reflections
are essential to achieve close alignment between the structures, then we conclude
that there is a chiral carbon in each molecule

![Fig. 1 Enantiomers prediction of CHFClBr with rotational-orthogonal Procrustes by comparing the atoms coordinates.](../examples/chirality_check/compounds.jpg "Fig. 1 Enantiomers prediction of CHFClBr with rotational-orthogonal Procrustes by comparing the atoms coordinates.")

The above figure shows enantiomers prediction of CHFClBr with rotational-orthogonal Procrustes by comparing the atoms coordinates. The molecule is shown in ball-stick representation with atoms labeled. The absolute configurations of the chiral atom are labelled in blue text.

In [6]:
# only run this cell if you need to install the dependices
!pip install git+https://github.com/theochem/procrustes.git@master

The 3D atomic coordinates have been extracted into `R.dat` and `S.dat`.

In [1]:
# load the libraries
import numpy as np
import matplotlib.pyplot as plt

from procrustes import rotational

In [2]:
# load the atomic coordinates
A = np.loadtxt("../examples/chirality_check/R.dat")
B = np.loadtxt("../examples/chirality_check/S.dat")

Now the molecule A is reflected with the reflection matrix. 

In [3]:
reflection = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
# create the reflection of compound A over the yz plane
A_ref = np.dot(A, reflection)

Now we compare the rms distance between structure when only rotational transformations are allowed and when both rotational transformations and reflection are allowed. 

In [4]:
res = rotational(A, B,
                 translate=True,
                 scale=False,
                 remove_zero_col=False,
                 remove_zero_row=False)
# Compute the error: reflection + rotation
res_ref = rotational(A_ref, B,
                     translate=True,
                     scale=False,
                     remove_zero_col=False,
                     remove_zero_row=False)

if res["error"]/res_ref["error"] > 10:
    print("These two compounds are enantiomers " 
          "and there is at least one chiral center in each of them.")
else:
    print("These two compounds are not enantiomers "
          "and there is no chiral center in any of them.")

These two compounds are enantiomers and there is at least one chiral center in each of them.


We can also wrap up everything we just done into a function.

In [ ]:
def chiral_check(A_coords, B_coords):
    r"""Check if a organic compound is chiral.

    Parameters
    ----------
    A_coords : string
        Atomic coordinates of the first organic compound A.
    B_coords : string
        Atomic coordinates of the first organic compound B.
    Returns
    -------
    A : ndarray
        3D coordinates of the first organic compound A.
    B : ndarray
        3D coordinates of the first organic compound B.
    """

    reflection = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
    # create the reflection of compound A over the yz plane
    A_ref = np.dot(A_coords, reflection)
    # Compute the rotational procrustes
    res = rotational(A_coords, B_coords,
                    translate=True,
                    scale=False,
                    remove_zero_col=False,
                    remove_zero_row=False)
    # Compute the error: reflection + rotation
    res_ref = rotational(A_ref, B_coords,
                        translate=True,
                        scale=False,
                        remove_zero_col=False,
                        remove_zero_row=False)

    if res["error"] / res_ref["error"] > 10:
        print("These two compounds are enantiomers "
              "and there is at least one chiral center in each of them.")
    else:
        print("These two compounds are not enantiomers "
              "and there is no chiral center in any of them.")

Some people might wonder how we can extract the coordinates from sdf files. Here are tthree options, one is to use `RDKit`, one is to use `iodata`, and the other is just by text processing.

In [ ]:
# only run this cell if you need to install the dependices
!conda install -c conda-forge rdkit

In [ ]:
# use rdkit to extract coordinates
from rdkit import Chem

def atom_coordinates_rdkit(sdf_name):
    r"""Load atomic coordinates from a sdf file with RDKit.
    
    Parameters
    ----------
    sdf_name : string
        SDF file name.
    
    Returns
    -------
    coords : ndarray
        3D atomic coordinates.
    """
    mol = Chem.SDMolSupplier(sdf_name)[0]
    conf = mol.GetConformer()
    return conf.GetPositions()

In [ ]:
# only run this cell if you need to install the dependices
!pip install git+https://github.com/theochem/iodata.git@master

In [ ]:
# use iodata to extract coordinates
# https://github.com/theochem/iodata
from iodata import load_one

def atom_coordinates_iodata(sdf_name):
    r"""Load atomic coordinates from a sdf file with iodata.
    
    Parameters
    ----------
    sdf_name : string
        SDF file name.
    
    Returns
    -------
    coords : ndarray
        3D atomic coordinates.
    """
    mol = load_one(sdf_name)
    coords = mol.atcoords
    return coords
    

In [ ]:
# use file processing to extract coordinates

def extract_coordinates(sdf_name):
    r"""Extract atomic coordinates from a sdf file.
    
    Parameters
    ----------
    sdf_name : string
        SDF file name.
    
    Returns
    -------
    coords : ndarray
        3D atomic coordinates.
    """
    coordinates = []
    with open(sdf_name, "r") as f:
        for line in f:
            line = line.strip()
            if line.endswith("V2000") or line.endswith("V3000"):
                break
        for line in f:
            line_seg = line.strip().split()
            if len(line_seg) == 10:
                coordinates.append(line_seg[:3])
            else:
                break
    coordinates = np.array(coordinates).astype(np.float)
    return coordinates